In [70]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType
from pyspark.sql.functions import *
from pyspark.sql.window import *
import random
from pyspark.sql import Row
# Criando uma sessão Spark
spark = SparkSession.builder.appName("SQL").getOrCreate()

# Criando DataFrame de clientes
schema_clientes = StructType([
    StructField('cliente_id', IntegerType(), True),
    StructField('nome', StringType(), True),
    StructField('idade', IntegerType(), True),
    StructField('cidade', StringType(), True),
    StructField('genero', StringType(), True)
])

clientes_data = [
    (i, 'Cliente' + str(i), random.randint(18, 65),
     random.choice(['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Brasília']),
     random.choice(['M', 'F']))
    for i in range(1, 10000)
]

df_clientes = spark.createDataFrame(clientes_data, schema=schema_clientes)

# Criando DataFrame de produtos
schema_produtos = StructType([
    StructField('produto_id', IntegerType(), True),
    StructField('nome_produto', StringType(), True),
    StructField('categoria', StringType(), True),
    StructField('preco', FloatType(), True)
   
])

produtos_data = [
    Row(
        produto_id=i,
        nome_produto='Produto' + str(i),
        categoria=random.choice(['Eletrônicos', 'Roupas', 'Alimentos', 'Acessórios']),
        preco=random.uniform(10, 10000),  # Convertendo para float
        disponibilidade=random.choice([True, False])
    )
    for i in range(1, 10000)
]

# Criando o DataFrame de produtos
df_produtos = spark.createDataFrame(produtos_data)

# # Criando DataFrame de vendas
schema_vendas = StructType([
    StructField('venda_id', IntegerType(), True),
    StructField('cliente_id', IntegerType(), True),
    StructField('produto_id', IntegerType(), True),
    StructField('quantidade', IntegerType(), True),
    StructField('data_venda', StringType(), True)
])

vendas_data = [
    (i, random.randint(1, 10000), random.randint(1, 50), random.randint(1, 5),  random.choice(['2023-01-01', '2022-01-01', '2021-01-01']))
    for i in range(1, 10000)
]

df_vendas = spark.createDataFrame(vendas_data, schema=schema_vendas)

# Criando DataFrame de transações
schema_transacoes = StructType([
    StructField('transacao_id', IntegerType(), True),
    StructField('produto_id', IntegerType(), True),
    StructField('valor', FloatType(), True),
    StructField('tipo', StringType(), True),
    StructField('status', StringType(), True)
])

transacoes_data = [
    (i, random.randint(1, 50), random.uniform(50, 500),
     random.choice(['Débito', 'Crédito']), random.choice(['Aprovado', 'Pendente', 'Rejeitado']))
    for i in range(1, 10000)
]

df_transacoes = spark.createDataFrame(transacoes_data, schema=schema_transacoes)

In [71]:
# Criando visões temporárias para os DataFrames
df_clientes.createOrReplaceTempView("clientes")
df_vendas.createOrReplaceTempView("vendas")
df_produtos.createOrReplaceTempView("produtos")
df_transacoes.createOrReplaceTempView("transacoes")

In [78]:
query = """
   WITH VendasValidadas AS (
    SELECT
        v.venda_id,
        v.cliente_id,
        v.produto_id,
        v.quantidade,
        v.data_venda,
        t.status AS status_transacao,
        ROW_NUMBER() OVER(PARTITION BY v.cliente_id ORDER BY v.data_venda DESC) AS rn
    FROM
        vendas v
    LEFT JOIN
        transacoes t ON v.produto_id = t.produto_id
    WHERE
        v.data_venda LIKE '2023%'
        AND t.status IN ('Aprovado', 'Pendente')
        AND t.status NOT IN ('Rejeitado')
),
VendasClientes AS (
    SELECT
        c.cliente_id,
        c.nome AS nome_cliente,
        c.idade,
        COUNT(vv.venda_id) AS total_vendas,
        CASE
            WHEN c.idade > 30 THEN 'Maior que 30'
            ELSE '30 ou Menos'
        END AS faixa_etaria
    FROM
        clientes c
    LEFT JOIN
        VendasValidadas vv ON c.cliente_id = vv.cliente_id
    WHERE
        c.cidade LIKE 'São%'
    GROUP BY
        c.cliente_id, c.nome, c.idade
    HAVING
        COUNT(vv.venda_id) > 2
),
TopClientes AS (
    SELECT
        cliente_id,
        nome_cliente,
        idade,
        total_vendas,
        faixa_etaria,
        ROW_NUMBER() OVER(ORDER BY total_vendas DESC) AS ranking
    FROM
        VendasClientes
)
SELECT
    tc.cliente_id,
    tc.nome_cliente,
    tc.idade,
    tc.total_vendas,
    tc.faixa_etaria,
    tc.ranking,
    ROUND(SUM(p.preco),2) AS valor_total_compras
FROM
    TopClientes tc
JOIN
    VendasValidadas vv ON tc.cliente_id = vv.cliente_id
JOIN
    produtos p ON vv.produto_id = p.produto_id
GROUP BY
    tc.cliente_id, tc.nome_cliente, tc.idade, tc.total_vendas, tc.faixa_etaria, tc.ranking
HAVING
    SUM(p.preco) > 1
ORDER BY
    tc.ranking;

"""
result = spark.sql(query)
result.show()

23/12/11 07:56:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 0

+----------+------------+-----+------------+------------+-------+-------------------+
|cliente_id|nome_cliente|idade|total_vendas|faixa_etaria|ranking|valor_total_compras|
+----------+------------+-----+------------+------------+-------+-------------------+
|      2998| Cliente2998|   24|         440| 30 ou Menos|      1|         2881088.54|
|      9446| Cliente9446|   58|         423|Maior que 30|      2|          2389789.4|
|      7726| Cliente7726|   52|         414|Maior que 30|      3|         1756960.21|
|      2608| Cliente2608|   18|         409| 30 ou Menos|      4|          915772.41|
|      5334| Cliente5334|   26|         409| 30 ou Menos|      5|         2357700.32|
|      9760| Cliente9760|   59|         406|Maior que 30|      6|         2471064.96|
|      9963| Cliente9963|   30|         406| 30 ou Menos|      7|         2009963.09|
|      2805| Cliente2805|   55|         405|Maior que 30|      8|         1799336.93|
|      6215| Cliente6215|   26|         399| 30 ou Men

23/12/11 07:56:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


##### Valid Sales

In [73]:
spec_windown = Window.partitionBy(col("cliente_id")).orderBy(col("data_venda").desc())
df_sales_2023 = df_vendas.filter(col("data_venda").like("2023%"))
df_valid_transactions = df_transacoes.filter((col("status").isin(["Aprovado", "Pendente"])) & (~col("status").isin(["Rejeitado"])))

df_valid_sales = df_sales_2023.alias("v").join(
    df_valid_transactions.alias("t"), on=col("v.produto_id") == col("t.produto_id"), how="left")\
    .select(
        "v.venda_id",
        "v.cliente_id",
        "v.produto_id",
        "v.quantidade",
        "v.data_venda",
        col("t.status").alias("status_transacao"),
        row_number().over(spec_windown).alias("rn")
    )

##### Customer Sales

In [74]:
df_customers_sp = df_clientes\
    .filter(col("cidade").like("São%"))\
    .withColumn("faixa_etaria", when(col("idade") > 30, "Maior que 30").otherwise("30 ou menos"))

df_customer_sales = df_customers_sp.alias("c").join(
    df_valid_sales.alias("v"), on=col("c.cliente_id") == col("v.cliente_id"), how="left")\
    .groupBy("c.cliente_id", "c.nome", "c.idade", "faixa_etaria")\
    .agg(count("v.venda_id").alias("total_vendas"))\
    .filter("total_vendas > 2")

##### Top Customers

In [75]:
spec = Window.orderBy(col("total_vendas").desc())
df_top_customers = df_customer_sales.withColumn("ranking",row_number().over(spec))

#### Final Result

In [80]:
df_top_customers.alias("c")\
    .join(df_valid_sales.alias("vv"), on=col("c.cliente_id") == col("vv.cliente_id"))\
    .join(df_produtos.alias("p"), on=col("p.produto_id") == col("vv.produto_id"))\
    .groupBy("c.cliente_id", "c.nome", "c.idade", "c.total_vendas", "c.faixa_etaria", "c.ranking")\
    .agg(round(sum(col("p.preco")),2).alias("valor_total_compras"))\
    .filter("valor_total_compras > 1")\
    .orderBy("ranking")\
    .show()

23/12/11 07:56:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 0

+----------+-----------+-----+------------+------------+-------+-------------------+
|cliente_id|       nome|idade|total_vendas|faixa_etaria|ranking|valor_total_compras|
+----------+-----------+-----+------------+------------+-------+-------------------+
|      2998|Cliente2998|   24|         440| 30 ou menos|      1|         2881088.54|
|      9446|Cliente9446|   58|         423|Maior que 30|      2|          2389789.4|
|      7726|Cliente7726|   52|         414|Maior que 30|      3|         1756960.21|
|      2608|Cliente2608|   18|         409| 30 ou menos|      4|          915772.41|
|      5334|Cliente5334|   26|         409| 30 ou menos|      5|         2357700.32|
|      9760|Cliente9760|   59|         406|Maior que 30|      6|         2471064.96|
|      9963|Cliente9963|   30|         406| 30 ou menos|      7|         2009963.09|
|      2805|Cliente2805|   55|         405|Maior que 30|      8|         1799336.93|
|      6215|Cliente6215|   26|         399| 30 ou menos|      9| 

23/12/11 07:56:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/11 07:56:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
